In [ ]:
#@title Friend's FertTest DIFF
from pathlib import Path
import zlib, re, random

try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

SIG = b"\x78\x9C"

# 29-byte building IDs to swap
REPAIR   = b"Building.Special.RepairmanHut"
TIN      = b"Building.Production.Mines.Tin"
COMPOST  = b"Building.Production.Composter"
assert len(REPAIR) == len(TIN) == len(COMPOST) == 29

# Composter to expect
NEEDLES = [b"Building.Production.Composter", b"Composter", b"CompostFacility", b"Compost"]

def upload(prompt):
  print(prompt)
  return files.upload() if IN_COLAB else {}

def iter_chunks(blob: bytes):
  pos=0
  while True:
    i = blob.find(SIG, pos)
    if i == -1: break
    d = zlib.decompressobj()
    dec = d.decompress(blob[i:])
    used = len(blob[i:]) - len(d.unused_data)
    if used <= 0: pos = i+2; continue
    comp = blob[i:i+used]
    yield (i, used, dec, comp)
    pos = i + used

def chunks(blob): return list(iter_chunks(blob))

def grep_counts_offsets(blob: bytes, needles):
  cnts = {n:0 for n in needles}
  offs = {n:[] for n in needles}
  for off,used,dec,comp in chunks(blob):
    for n in needles:
      c = dec.count(n)
      if c:
        cnts[n]+=c; offs[n].append(off)
  return cnts, offs

def ascii_strings(b: bytes, minlen=6):
  return [m.group(0) for m in re.finditer(rb"[ -~]{%d,}"%minlen, b)]

def ub_chunk(blob: bytes):
  for off,used,dec,comp in chunks(blob):
    if b"UnlockedBuildings" in dec:
      return off,used,dec,comp
  return None

def best_recipe_like_chunk(blob: bytes):
  """Pick the user's chunk most likely holding availability/recipe flags."""
  best=None; score=-1
  for off,used,dec,comp in chunks(blob):
    s = 0
    s += 10*dec.count(b"Recipe")
    s += 10*dec.count(b"Available")
    s += 5*dec.count(b"Production")
    s += 2*dec.count(b"Build")
    s += 1*dec.count(b"Compost")
    if s > score:
      score = s; best = (off,used,dec,comp,s)
  return best

def build_equal_len_donor(dec: bytes, target: bytes, avoid=set(), prefer_prefix=b"Building."):
  L = len(target)
  for s in ascii_strings(dec, 10):
    if len(s)==L and s.startswith(prefer_prefix) and (s not in avoid) and (b"Compost" not in s):
      return s
  for s in ascii_strings(dec, 6):
    if len(s)==L and (s not in avoid) and s.isalnum():
      return s
  return None

def brute_compress_match_len(payload: bytes, target_len: int):
  for level in range(0,10):
    for mem in range(1,10):
      for strat in (0,1,2,3,4):
        co = zlib.compressobj(level, zlib.DEFLATED, 15, mem, strat)
        out = co.compress(payload) + co.flush()
        if len(out) == target_len:
          return out
  return None

def safe_toggle_positions(payload: bytes, protect_ranges, budget=600):
  safe=[]
  text = payload.replace(b"\x00", b"\n")
  for m in re.finditer(rb"[ -~]{24,}", text):
    s,e = m.span()
    for i in range(s, min(e, s+1000)):
      ch = text[i]
      if not (65<=ch<=90 or 97<=ch<=122): continue
      bad=False
      for a,b in protect_ranges:
        if a<=i<b: bad=True; break
      if not bad: safe.append(i)
  random.shuffle(safe)
  return safe[:budget]

def preserve_len_compress(new_dec: bytes, orig_len: int, protect_tokens=[b"Building.", b"Compost", b"Unlocked", b"Recipe", b"Available", b"Fertility"]):
  c = brute_compress_match_len(new_dec, orig_len)
  if c: return c
  pr=[]
  for tok in protect_tokens:
    start=0
    while True:
      j = new_dec.find(tok, start)
      if j==-1: break
      pr.append((max(0,j-20), min(len(new_dec), j+len(tok)+20)))
      start = j+1
  buf = bytearray(new_dec)
  tries=0
  for idx in safe_toggle_positions(new_dec, pr):
    ch = buf[idx]
    buf[idx] = ch+32 if 65<=ch<=90 else (ch-32 if 97<=ch<=122 else ch)
    tries+=1
    c = brute_compress_match_len(bytes(buf), orig_len)
    if c: return c
    buf[idx] = ch
    if tries>=400: break
  return None

def rebuild_one_chunk(blob: bytes, target_off: int, new_comp: bytes):
  out=bytearray(); pos=0
  for off,used,dec,comp in chunks(blob):
    out += blob[pos:off]
    out += new_comp if off==target_off else comp
    pos = off + used
  out += blob[pos:]
  return bytes(out)

# ======= Upload three files =======
print("👉 Upload your WORKING save (the one that loads your city).")
u1 = upload("Select Level.sav (required)")
if not u1: raise SystemExit("No file uploaded.")
me_name = next((k for k in u1 if k.lower().endswith(".sav")), None)
me = u1[me_name]; Path(me_name).write_bytes(me)
print(f"Saved: {me_name} ({len(me):,} bytes)")

print("👉 Upload Level_verified_friend_got_it_right.sav (optional), else Cancel")
u2 = upload("Select friend save (optional)")
friend = None
if u2:
  fn = next((k for k in u2 if k.lower().endswith(".sav")), None)
  if fn: friend = u2[fn]; Path(fn).write_bytes(friend); print(f"Saved: {fn} ({len(friend):,} bytes)")

print("👉 Upload Level_fert_test.sav (optional), else Cancel")
u3 = upload("Select fert test (optional)")
fert = None
if u3:
  ft = next((k for k in u3 if k.lower().endswith(".sav")), None)
  if ft: fert = u3[ft]; Path(ft).write_bytes(fert); print(f"Saved: {ft} ({len(fert):,} bytes)")

if friend and fert:
  def strset(blob):
    s=set()
    for _,_,dec,_ in chunks(blob):
      for ss in ascii_strings(dec, 8):
        if any(t in ss.lower() for t in [b"compost", b"composter", b"compostfacility"]):
          s.add(ss)
    return s
  frs = strset(friend); fts = strset(fert); mys = strset(me)
  inter = frs & fts
  missing = [x.decode('latin1','ignore') for x in sorted(inter - mys, key=lambda v:(len(v),v))]
  print("\n[friend ∩ fert_test] − yours (strings you lack):")
  for s in missing[:80]: print("  •", s)

# Baseline counts
cnts_before, offs_before = grep_counts_offsets(me, NEEDLES)
print("\n[Your BEFORE] compost family:")
for n in NEEDLES:
  print(f"  {n.decode('latin1','ignore'):>28s}: {cnts_before[n]}  (chunks: {offs_before[n]})")

# Site 1: UnlockedBuildings swap(s) (29-byte donors) — length-preserved
ub = ub_chunk(me)
if not ub: raise SystemExit("UnlockedBuildings chunk not found in your save.")
uoff, uused, udec, ucomp = ub
orig_len = len(ucomp)
print(f"\nUnlockedBuildings chunk off={uoff} comp_len={orig_len}")

def ub_apply_swaps(dec):
  out = dec
  swaps = []
  if REPAIR in out and COMPOST not in out:
    out = out.replace(REPAIR, COMPOST, 1); swaps.append((REPAIR, COMPOST))
  if TIN in out and COMPOST not in out:
    out = out.replace(TIN, COMPOST, 1); swaps.append((TIN, COMPOST))
  return out, swaps

dec1, swaps1 = ub_apply_swaps(udec)
if swaps1:
  comp1 = preserve_len_compress(dec1, orig_len)
  if comp1:
    me_site1 = rebuild_one_chunk(me, uoff, comp1)
    print(f"Site1 OK. Swaps: {[s[0].decode('latin1','ignore') for s in swaps1]} → Composter  (len preserved={orig_len})")
  else:
    print("Site1 could not preserve length — leaving UB unchanged.")
    me_site1 = me
else:
  print("Site1: UB already had Composter or no donors found; skipping.")
  me_site1 = me

# Site 2: Recipe/Available chunk — inject 'Composter' and 'CompostFacility' (equal-length donors), len-preserved
target_tokens = [b"Composter", b"CompostFacility"]
off2,used2,dec2,comp2,score = best_recipe_like_chunk(me_site1)
print(f"\nRecipe/Available-like chunk off={off2} comp_len={len(comp2)} score={score}")
dec2_new = dec2
swaps2=[]

for token in target_tokens:
  if token in dec2_new:
    continue  # already present here
  donor = build_equal_len_donor(dec2_new, token, avoid=set([COMPOST, REPAIR, TIN]))
  if donor and (len(donor)==len(token)):
    dec2_new = dec2_new.replace(donor, token, 1)
    swaps2.append((donor, token))

if swaps2:
  comp2_new = preserve_len_compress(dec2_new, len(comp2))
  if comp2_new:
    me_site2 = rebuild_one_chunk(me_site1, off2, comp2_new)
    print("Site2 OK. Injected tokens (len-preserved):", [(d.decode('latin1','ignore'), t.decode('latin1','ignore')) for d,t in swaps2])
  else:
    print("Site2 could not preserve compressed length — leaving chunk unchanged.")
    me_site2 = me_site1
else:
  print("Site2: No suitable equal-length donors in recipe-like chunk; unchanged.")
  me_site2 = me_site1

#  AFTER report & outputs
cnts_after, offs_after = grep_counts_offsets(me_site2, NEEDLES)
print("\n[Your AFTER] compost family:")
for n in NEEDLES:
  print(f"  {n.decode('latin1','ignore'):>28s}: {cnts_after[n]}  (chunks: {offs_after[n]})")

outA = "Level_patched_Composter_siteB_lenPreserved.sav"
outB = "Level_patched_addComposter_lenPreserved_twoSites.sav"
Path(outA).write_bytes(me_site2)
Path(outB).write_bytes(me_site2)
print("\nWrote:")
print(" ", outA, f"({len(me_site2):,} bytes)")
print(" ", outB, f"({len(me_site2):,} bytes)")

if IN_COLAB:
  try: files.download(outA)
  except: pass
  try: files.download(outB)
  except: pass

print("\nDone.")


👉 Upload your WORKING save (the one that loads your city).
Select Level.sav (required)


Saving Level.sav to Level.sav
Saved: Level.sav (12,917,948 bytes)
👉 Upload Level_verified_friend_got_it_right.sav (optional), else Cancel
Select friend save (optional)


Saving Level_verified_friend_got_it_right.sav to Level_verified_friend_got_it_right.sav
Saved: Level_verified_friend_got_it_right.sav (12,874,418 bytes)
👉 Upload Level_fert_test.sav (optional), else Cancel
Select fert test (optional)


Saving Level_fert_test.sav to Level_fert_test.sav
Saved: Level_fert_test.sav (13,093,287 bytes)

[friend ∩ fert_test] − yours (strings you lack):
  • Composter
  • CompostFacility
  • Building.Production.Composter

[Your BEFORE] compost family:
  Building.Production.Composter: 0  (chunks: [])
                     Composter: 0  (chunks: [])
               CompostFacility: 0  (chunks: [])
                       Compost: 80  (chunks: [4153965, 12682818, 12764337, 12769971, 12774958, 12825218, 12887413, 12895705, 12907215])

UnlockedBuildings chunk off=12682818 comp_len=23595
Site1 OK. Swaps: ['Building.Special.RepairmanHut'] → Composter  (len preserved=23595)

Recipe/Available-like chunk off=4143299 comp_len=3587 score=4298
Site2 OK. Injected tokens (len-preserved): [('CopperOre', 'Composter')]

[Your AFTER] compost family:
  Building.Production.Composter: 1  (chunks: [12682818])
                     Composter: 1  (chunks: [12682818])
               CompostFacility: 0  (chunks: [])
      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Done.
